# Выводы: 


# Домашнее задание. Поиск ассоциативных правил
**Задача**: Ваша исследовательская задача - понять, есть ли причинно-следственные связи между диагнозами сердечных заболеваний.


**Данные**: https://archive.ics.uci.edu/ml/datasets/spect+heart . Данные содержат 22 бинарных признака - 21 вспомогательный диагноз и 1 финальный для каждого пациента




In [ ]:
import pandas as pd

In [ ]:
df_test = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/spect/SPECT.test');
df_train = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/spect/SPECT.train');

In [ ]:
df_train.head()

,1,0,0.1,0.2,1.1,0.3,0.4,0.5,1.2,1.3,0.6,0.7,0.8,1.4,1.5,0.9,0.10,0.11,0.12,0.13,0.14,0.15,0.16
0,1,0,0,1,1,0,0,0,1,1,0,0,0,1,1,0,0,0,0,0,0,0,1
1,1,1,0,1,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,1
3,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,1,0,0,0,0,0,0
4,1,0,0,0,1,0,0,0,0,1,0,0,0,1,1,0,1,0,0,0,1,0,1


В качестве алгоритма для поиска ассоциативных правил рассмотрим FP-деревья, они являются оптимизацией Apriori

In [ ]:
pip install pyfpgrowth

In [ ]:
import pyfpgrowth
patterns = pyfpgrowth.find_frequent_patterns(df_train, 2) # найдём все правила, которые встречаются не менее двух раз
rules = pyfpgrowth.generate_association_rules(patterns, 0.30) # найдём все следствия из правил, которые происходят не реже чем в 70% случаев

In [ ]:
patterns

{('.',): 21,
 ('.', '0'): 17,
 ('.', '0', '1'): 10,
 ('.', '0', '2'): 2,
 ('.', '0', '3'): 2,
 ('.', '0', '4'): 2,
 ('.', '0', '5'): 2,
 ('.', '0', '6'): 2,
 ('.', '1'): 15,
 ('.', '1', '1'): 2,
 ('.', '1', '2'): 2,
 ('.', '1', '3'): 2,
 ('.', '1', '4'): 2,
 ('.', '1', '5'): 2,
 ('.', '2'): 3,
 ('.', '3'): 3,
 ('.', '4'): 3,
 ('.', '5'): 3,
 ('.', '6'): 2,
 ('0',): 18,
 ('0', '1'): 10,
 ('0', '2'): 2,
 ('0', '3'): 2,
 ('0', '4'): 2,
 ('0', '5'): 2,
 ('0', '6'): 2,
 ('1', '1'): 2,
 ('1', '2'): 2,
 ('1', '3'): 2,
 ('1', '4'): 2,
 ('1', '5'): 2,
 ('6',): 2}

In [ ]:
rules

{('.',): (('0',), 0.8095238095238095),
 ('.', '0'): (('1',), 0.5882352941176471),
 ('.', '1'): (('0',), 0.6666666666666666),
 ('.', '2'): (('1',), 0.6666666666666666),
 ('.', '3'): (('1',), 0.6666666666666666),
 ('.', '4'): (('1',), 0.6666666666666666),
 ('.', '5'): (('1',), 0.6666666666666666),
 ('.', '6'): (('0',), 1.0),
 ('0',): (('.',), 0.9444444444444444),
 ('0', '1'): (('.',), 1.0),
 ('0', '2'): (('.',), 1.0),
 ('0', '3'): (('.',), 1.0),
 ('0', '4'): (('.',), 1.0),
 ('0', '5'): (('.',), 1.0),
 ('0', '6'): (('.',), 1.0),
 ('1', '1'): (('.',), 1.0),
 ('1', '2'): (('.',), 1.0),
 ('1', '3'): (('.',), 1.0),
 ('1', '4'): (('.',), 1.0),
 ('1', '5'): (('.',), 1.0),
 ('6',): (('.', '0'), 1.0)}

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
def transaction_list(df):
    list_external=[]
    for i in range(df.shape[0]):
        list_internal=[]
        data=df.iloc[i]
        index=data[data>0]
        for element in index.index:
            list_internal.append(element)
        list_external.append(list_internal)
    return list_external
  
df_train_matrix = transaction_list(df_train); 

In [ ]:
df_train_matrix[0]

['1', '0.2', '1.1', '1.2', '1.3', '1.4', '1.5', '0.16']

In [ ]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from datetime import datetime

In [ ]:
te = TransactionEncoder()
te_ary = te.fit(df_train_matrix).transform(df_train_matrix)
df = pd.DataFrame(te_ary, columns=te.columns_)
start = datetime.now()
apriori_itemsets = apriori(df, min_support=0.2, use_colnames=True)
finish = datetime.now()
print(finish-start)

0:00:00.021385


In [ ]:
rules = association_rules(apriori_itemsets, metric="lift", min_threshold=2)
rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))
rules = rules[(rules['lift']>=2) & (rules['confidence']>=0.75)]

In [ ]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
0,(0),(0.3),0.367089,0.303797,0.303797,0.827586,2.724138,0.192277,4.037975,1
1,(0.3),(0),0.303797,0.367089,0.303797,1.000000,2.724138,0.192277,inf,1
2,(0),(0.6),0.367089,0.291139,0.291139,0.793103,2.724138,0.184265,3.426160,1
3,(0.6),(0),0.291139,0.367089,0.291139,1.000000,2.724138,0.184265,inf,1
4,(0.2),(1.2),0.265823,0.265823,0.215190,0.809524,3.045351,0.144528,3.854430,1
5,(1.2),(0.2),0.265823,0.265823,0.215190,0.809524,3.045351,0.144528,3.854430,1
6,(0.2),(1.4),0.265823,0.341772,0.227848,0.857143,2.507937,0.136997,4.607595,1
8,(0.6),(0.3),0.291139,0.303797,0.240506,0.826087,2.719203,0.152059,4.003165,1
9,(0.3),(0.6),0.303797,0.291139,0.240506,0.791667,2.719203,0.152059,3.402532,1
10,(0.5),(0.8),0.265823,0.253165,0.215190,0.809524,3.197619,0.147893,3.920886,1


In [ ]:
!pip install mlxtend --upgrade --no-deps
import mlxtend
print(mlxtend.__version__)
from mlxtend.frequent_patterns import fpgrowth
start = datetime.now()
fpgrowth_itemsets = fpgrowth(df, min_support=0.2, use_colnames=True)
finish = datetime.now()
print(finish-start)

Requirement already up-to-date: mlxtend in /usr/local/lib/python3.6/dist-packages (0.17.3)
0.17.3
0:00:00.010122


In [ ]:
rules = association_rules(fpgrowth_itemsets, metric="lift", min_threshold=2)
rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))
rules = rules[(rules['lift']>=2) & (rules['confidence']>=0.75)]
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
0,(1.2),(1.4),0.265823,0.341772,0.202532,0.761905,2.229277,0.111681,2.764557,1
2,(0.2),(1.4),0.265823,0.341772,0.227848,0.857143,2.507937,0.136997,4.607595,1
4,(0.2),(1.2),0.265823,0.265823,0.215190,0.809524,3.045351,0.144528,3.854430,1
5,(1.2),(0.2),0.265823,0.265823,0.215190,0.809524,3.045351,0.144528,3.854430,1
6,(0),(0.3),0.367089,0.303797,0.303797,0.827586,2.724138,0.192277,4.037975,1
7,(0.3),(0),0.303797,0.367089,0.303797,1.000000,2.724138,0.192277,inf,1
8,(0),(0.6),0.367089,0.291139,0.291139,0.793103,2.724138,0.184265,3.426160,1
9,(0.6),(0),0.291139,0.367089,0.291139,1.000000,2.724138,0.184265,inf,1
10,(0.6),(0.3),0.291139,0.303797,0.240506,0.826087,2.719203,0.152059,4.003165,1
11,(0.3),(0.6),0.303797,0.291139,0.240506,0.791667,2.719203,0.152059,3.402532,1
